In [26]:
# !pip install torch
# !pip install transformers

In [1]:
import json
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, BertConfig
import torch
import torch.nn as nn

/Users/nalishjain/Acad Sem 6/NLP-Assignments/A4_16/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
!pip show torch

Name: torch
Version: 2.2.2
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /Users/nalishjain/Acad Sem 6/NLP-Assignments/A4_16/.venv/lib/python3.10/site-packages
Requires: filelock, fsspec, jinja2, networkx, sympy, typing-extensions
Required-by: 


In [2]:
with open("train_file.json", "r") as train_file:
    train_data = json.load(train_file)

with open("val_file.json", "r") as val_file:
    val_data = json.load(val_file)

In [3]:
train_data[0]

{'episode': 'utterance_3492',
 'speakers': ['Phoebe', 'Eric', 'Phoebe', 'Eric', 'Phoebe'],
 'emotions': ['surprise', 'fear', 'surprise', 'sadness', 'disgust'],
 'utterances': ['You-you\x85you had sex with Ursula?!',
  'Uh, a little bit. She-she-she walked in and I thought she was you and I kissed her and',
  "You didn't notice she was wearing different clothes?!",
  'Well I was just so excited to see you.',
  "Oh. Ew! Ew! Ew! Ugh! Y'know what? This is too weird."],
 'triggers': [1.0, 1.0, 0.0, 0.0, 0.0]}

In [4]:
class EmbeddingBert(nn.Module):
    def __init__(self):
        super(EmbeddingBert, self).__init__()
        self.pre_trained = BertModel.from_pretrained("bert-base-uncased")
        # self.classifier =  nn.Linear(self.pre_trained.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask):
        x = self.pre_trained(input_ids = input_ids, attention_mask = attention_mask)
        x = x.pooler_output
        return x
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model_bert = EmbeddingBert()

In [5]:
def remove_duplicates(data):
    unique_data_list = []
    seen = set()
    for d in data:
        d_tuple = {k: tuple(v) if isinstance(v, list) else v for k, v in d.items()}
        t = tuple(sorted(d_tuple.items()))
        if t not in seen:
            unique_data_list.append(d)
            seen.add(t)
    return unique_data_list

device = torch.device('cuda')
def create_mapping(data, model):
    mapping = {}
    count = 0
    maxi = 0
    model = model.to(device)
    for dialogue in data:

        tokenised = tokenizer(dialogue['utterances'], return_tensors="pt", max_length = 100, padding="max_length", truncation=True, add_special_tokens = True)
        # print(count, "/", len(data),tokenised['input_ids'].shape[0])
        # print(tokenised['input_ids'].shape)
        # maxi = max(maxi, tokenised['input_ids'].shape[0])
        embeddings = []
        for k in range(0, tokenised['input_ids'].shape[0], 16):
          embedding = model(tokenised['input_ids'][k:k+16,:].to(device), tokenised['attention_mask'][k:k+16,:].to(device))
          embeddings.extend(embedding.detach().cpu().numpy())
          torch.cuda.empty_cache()
        # print(len(embeddings), len(dialogue['utterances']))
        mapping[count] = [dialogue['utterances'], embeddings, dialogue['emotions'], dialogue['triggers']]
        # break
        count += 1
    # print(maxi)
    return mapping

In [ ]:
unique_train_data = remove_duplicates(train_data)
unique_val_data = remove_duplicates(val_data)
train_dict  = create_mapping(unique_train_data, model_bert)
val_dict = create_mapping(unique_val_data, model_bert)


In [101]:
import pickle
with open('/content/drive/MyDrive/train_dict.pkl', 'wb') as f:
    pickle.dump(train_dict, f)

with open('/content/drive/MyDrive/val_dict.pkl', 'wb') as f:
    pickle.dump(val_dict, f)

808

In [6]:
unique_val_data_list = []
seen = set()
for d in val_data:
    d_tuple = {k: tuple(v) if isinstance(v, list) else v for k, v in d.items()}
    t = tuple(sorted(d_tuple.items()))
    if t not in seen:
        unique_val_data_list.append(d)
        seen.add(t)

print(len(unique_val_data_list))

808


In [111]:
len(train_data[1]['utterances'])

15

In [8]:
unique_val_data_list[0]['utterances']

['Hey! Hold on a minute, hold on a second. Do you think these pearls are nice?',
 "I'd really prefer a mountain bike.",
 "Janice's birthday is coming up, I want to get her something speacial. Come in here with me.",
 'Whoa, whoa, whoa, wait, whoa.',
 'Do you ah, want to get her something speacial, get her flowers, get her candy, get her gum, girls love gum.',
 "That's a good idea, \x91Dear Janice have a Hubba-Bubba birthday'. I would like to get her something serious.",
 'Oh, you want something serious.',
 "Y'know what you should do, you should get her one of those um, barium enemas.",
 'Those are dead serious.',
 "All right. Look, I'm gonna go in here, and you don't buy me anything ever.",
 "No, no, you can't, you can't, okay, you can't, you can't buy her pearls, you just can't, you can't, you can't.",
 'Why not?!',
 "Oh God. Uh, okay, here's the thing, this is the thing, okay, the thing is...",
 'What is the thing?',
 "Okay. I went down to the \x91Mattress King' showroom and, and I s